In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


In [2]:
def generate_query(endpoint_url, query, limit):

    # combine endpoint_url, query, and limit into a single string
    raw_query = (f"{endpoint_url}?$query="
                 f"{query}%20"
                 f"limit {limit}"
                )
    
    # get rid of control characters
    for replacements in ((" ", "%20"), ("\n", "%20")):
        raw_query = raw_query.replace(*replacements)
    
    return raw_query

In [3]:
endpoint_url = "https://data.cincinnati-oh.gov/resource/4cjh-bm8b.json"

query = """select date_extract_y(requested_date) as year,
                  address, 
                  count(*) as n
           where service_code IN ('"TLGR-PRV"','"LITR-PRV"','"DUMP-PVS"')
           and requested_date>='2019-01-01'
           and requested_date<'2023-01-01'
           group by address, year
           order by n desc
           """


query_url = generate_query(endpoint_url = endpoint_url,
                           query = query,
                           limit = 1000000)

In [4]:
requests_by_address = pd.read_json(query_url)

requests_by_address['year'] = requests_by_address['year'].astype('string')

In [5]:
requests_by_address['clean_address'] = requests_by_address.address.str.replace(', CINC', '', regex=False) \
    .str.split('-').str[0] \
    .str.strip() \
    .str.replace('"', '')

In [6]:
requests_by_address = requests_by_address \
    .groupby(['year','clean_address'], as_index=False) \
    .agg('sum') \
    .sort_values(by = 'n', ascending=False)

In [7]:
total_requests_by_year = requests_by_address \
    .groupby(['year']) \
    .agg(total_n= ('n', 'sum'))

In [8]:
pct_request_by_address = requests_by_address \
    .join(total_requests_by_year, on = "year") \
    .assign(pct_requests = lambda x: x['n'] / x['total_n'])

In [9]:
def get_percentile_df(df, year):
    
    requests_sorted = df.query(f"year == '{year}'") \
        .sort_values(by = 'pct_requests', ascending = False)
    
    requests_sorted['row_number'] = requests_sorted.reset_index().index
    
    total_rows = len(requests_sorted)
    
    requests_sorted['row_percentile'] = np.ceil(((requests_sorted['row_number'] + 1) / total_rows) * 100)
    
    agg_requests = requests_sorted \
        .groupby(['row_percentile'], as_index=False) \
        .aggregate({'pct_requests':'sum'})
    
    agg_requests['cumsum'] = np.round(agg_requests['pct_requests'].cumsum()*100, 2)
    
    agg_requests['year'] = year
    
    return agg_requests

In [10]:
key_percentiles = "1,5,10,20"

def get_key_percentiles(pct_request_by_address, year):
    
    percentiles = get_percentile_df(pct_request_by_address, year)
    
    key_percentiles = percentiles.query(f"row_percentile in (1,5,10,20)")
    
    return key_percentiles

In [11]:
years = [2019, 2020, 2021, 2022]

all_key_percentiles = get_key_percentiles(pct_request_by_address, 2020).query("year == 2019")

for i in years:
    
    year_key_percentiles = get_key_percentiles(pct_request_by_address, i)
    
    all_key_percentiles = pd.concat([all_key_percentiles, year_key_percentiles])
    
all_key_percentiles['year'] = all_key_percentiles['year'].astype('string')
    

In [18]:
percentile_val = "10"
fig = px.bar(all_key_percentiles.query(f"row_percentile == {percentile_val}"), 
             x="year", 
             y="cumsum", 
             text_auto=True,
             template='plotly_white',
             labels={
                     "year": "Year",
                     "cumsum": "Percent of Requests"                 
                   },
             title=f"Graph 2. Percentage of PLAP Requests Contributed by Top {percentile_val}% of Addresses"
            )
fig.show()

In [16]:
requests_sorted = pct_request_by_address.query(f"year == '2021'") \
    .sort_values(by = 'pct_requests', ascending = False)

requests_sorted['row_number'] = requests_sorted.reset_index().index

total_rows = len(requests_sorted)

requests_sorted['row_percentile'] = np.ceil(((requests_sorted['row_number'] + 1) / total_rows) * 100)

requests_sorted.query("row_percentile <= 10").describe()

fig = px.histogram(requests_sorted.query("row_percentile <= 1"), x = 'n', 
                   template='plotly_white',
                   labels={
                     "count": "Number of Addresses",
                     "n": "Number of Requests Experienced by an Address"                 
                   },
                title="Histogram of Request Volume for Top 1% of PLAP Requested Addresses in 2021",
                  text_auto=True)

fig.update_layout(yaxis_title="Count of Addresses")
fig.update_traces(textposition='outside')


fig.show()

In [22]:
requests_sorted

,year,clean_address,n,total_n,pct_requests,row_number,row_percentile
16564,2021,847 HUTCHINS AV,32,8431,0.003796,0,1.0
14881,2021,3973 LOWRY AV,27,8431,0.003202,1,1.0
11301,2021,1035 PARKSON PL,25,8431,0.002965,2,1.0
15214,2021,4460 MELLWOOD AV,20,8431,0.002372,3,1.0
14831,2021,3916 W LIBERTY ST,16,8431,0.001898,4,1.0
...,...,...,...,...,...,...,...
13975,2021,310 TERRACE AV,1,8431,0.000119,5697,100.0
13976,2021,3100 AUTEN AV,1,8431,0.000119,5698,100.0
13977,2021,3101 BERNINGHAUS AV,1,8431,0.000119,5699,100.0
13978,2021,3101 COSTELLO AV,1,8431,0.000119,5700,100.0


In [27]:
year = 2022
requests_sorted = pct_request_by_address.query(f"year == '{year}'") \
    .sort_values(by = 'pct_requests', ascending = False)

requests_sorted['row_number'] = requests_sorted.reset_index().index

total_rows = len(requests_sorted)

requests_sorted['row_percentile'] = np.ceil(((requests_sorted['row_number'] + 1) / total_rows) * 100)

pct_request_adjusted = requests_sorted \
    .assign(n = lambda x: np.where(x['row_percentile'] == 1, np.round(x['n'] / 2), x['n']))

adj_total = np.sum(pct_request_adjusted['n'])

pct_request_adjusted['pct_requests'] = pct_request_adjusted['n'] / adj_total

In [29]:
years = [2019, 2020, 2021, 2022]

all_key_percentiles_adj = get_key_percentiles(pct_request_adjusted, 2020).query("year == 2019")

for i in years:
    
    year_key_percentiles_adj = get_key_percentiles(pct_request_adjusted, i)
    
    all_key_percentiles_adj = pd.concat([all_key_percentiles_adj, year_key_percentiles_adj])
    
all_key_percentiles_adj['year'] = all_key_percentiles_adj['year'].astype('string')
    

In [30]:
all_key_percentiles_adj

,row_percentile,pct_requests,cumsum,year
0,1.0,0.040816,4.08,2022
4,5.0,0.025152,15.43,2022
9,10.0,0.018864,25.66,2022
19,20.0,0.013900,39.54,2022
